# 新浪财经资讯

In [1]:
import requests
import time
import csv
import codecs
import re
import random
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
from lxml import etree
import scrapy
from multiprocessing import cpu_count
from multiprocessing import Process, Queue

In [2]:
stock_id = pd.read_csv("stock_id.csv")

In [3]:
sz = stock_id[0: 1] #[0: 1384]
#hs = stock_id[2045: 3402] #[2045: 3402]

## 抓取深沪股市资讯列表

In [ ]:
def get_url():
    sz_url, hs_url = [], []
    for i in range(0, len(sz)):
        a = 'http://vip.stock.finance.sina.com.cn/corp/view/vCB_AllNewsStock.php?symbol=sz' + sz.SecuCode[i]
        sz_url.append(a)
    for j in range(2045, 3402):
        b = 'http://vip.stock.finance.sina.com.cn/corp/view/vCB_AllNewsStock.php?symbol=hs' + hs.SecuCode[j]
        sz_url.append(b)
    return sz_url

In [ ]:
GetUrls = get_url()

### 只爬取深市(测试用)

In [4]:
def get_sz_url():
    sz_url, SecuCode_list = [], []
    for i in range(0, len(sz)):
        a = 'http://vip.stock.finance.sina.com.cn/corp/view/vCB_AllNewsStock.php?symbol=sz' + sz.SecuCode[i]
        sz_url.append(a)
    return sz_url

In [5]:
GetUrls = get_sz_url()

In [ ]:
GetUrls

### 只爬取沪市(测试用)

In [ ]:
def get_hs_url():
    hs_url = []
    for j in range(2047, 3402):
        b = 'http://vip.stock.finance.sina.com.cn/corp/view/vCB_AllNewsStock.php?symbol=hs' + hs.SecuCode[j]
        hs_url.append(b)
    return hs_url,SecuCode_list

##  获取Stock_id

In [ ]:
def get_id():
    sz_list, hs_list = [], []
    for i in range(0, len(sz)):
        a = 'sz' + sz.SecuCode[i]
        Sz_SecuCode = re.findall("(sz\d\d\d\d\d\d)", a)
        sz_list.append(Sz_SecuCode)
    for j in range(2045, 3402):
        b = 'hs' + hs.SecuCode[j]
        Hs_SecuCode = re.findall("(hs\d\d\d\d\d\d)", b)       
        hs_list.append(Hs_SecuCode)
    sz_list.extend(hs_list)
    return sz_list

### 获取深市StockID(测试用)

In [6]:
def get_id():
    sz_list, hs_list = [], []
    for i in range(0, len(sz)):
        a = 'sz' + sz.SecuCode[i]
        Sz_SecuCode = re.findall("(sz\d\d\d\d\d\d)", a)
        sz_list.append(Sz_SecuCode)
    return sz_list

### 获取沪市StockID(测试用)

## 爬取新闻链接

In [7]:
from Download import request
def get_data_news_zhongxin():
    news_list, URL_list = [], []
    for i in range(0, len(GetUrls)):                             # 0 is first stock_id
        Pages = GetUrls[i]
        for j in range(0,1):                                     # catch page1 - 10 firstly，这里只抓了每个股的第一页
            r = request.get(str(Pages) + '&Page=' + str(j), 3)
            r.encoding='gb2312'
            selector = etree.HTML(r.text)
            infos = selector.xpath('//div[@class="datelist"]')
            for info in infos:
                URL = info.xpath('//td/div[1]/ul/a/@href')
                #print(URL)
                URL_list.extend(URL)
    return URL_list

In [8]:
gdnz = get_data_news_zhongxin() ###所有的新闻链接

In [9]:
gdnz

['http://finance.sina.com.cn/stock/s/2018-09-14/doc-ihkahyhw8150416.shtml',
 'http://finance.sina.com.cn/chanjing/gsnews/2018-09-11/doc-ihiixyeu6288005.shtml',
 'http://finance.sina.com.cn/roll/2018-09-10/doc-ihivtsyk8575881.shtml',
 'http://finance.sina.com.cn/chanjing/gsnews/2018-09-10/doc-ihivtsyk8131280.shtml',
 'http://finance.sina.com.cn/chanjing/gsnews/2018-09-08/doc-ihivtsyi6893086.shtml',
 'http://finance.sina.com.cn/roll/2018-09-04/doc-ihiqtcap3371000.shtml',
 'http://finance.sina.com.cn/roll/2018-09-04/doc-ihiqtcan8351044.shtml',
 'http://finance.sina.com.cn/roll/2018-09-04/doc-ihiqtcan8059637.shtml',
 'http://finance.sina.com.cn/stock/s/2018-09-04/doc-ihiqtcan7724278.shtml',
 'http://finance.sina.com.cn/stock/s/2018-08-28/doc-ihifuvpi0153076.shtml',
 'http://finance.sina.com.cn/roll/2018-08-28/doc-ihifuvpi0104918.shtml',
 'http://finance.sina.com.cn/stock/s/2018-08-27/doc-ihifuvph3775430.shtml',
 'http://finance.sina.com.cn/stock/gujiayidong/2018-08-24/doc-ihicsiaw3191805.s

### 保存爬取的新闻链接至csv（方便以后爬取作去重）

In [11]:
###保存urls
for i in range(0, len(gdnz)):
    a = gdnz[i]
    with open('sina_info_url.csv', 'a', newline='') as csvfile:
        w = csv.writer(csvfile)
        #print(w)
        w.writerow([a]) ### 我就改了这个地方！！！！ 原本只能输出一行的， 现在终于可以一个一列了！

## 将文本与SrockID合并并输出

### 输出StockID至list

In [12]:
gsid = get_id()

In [13]:
gsid

[['sz000001']]

In [14]:
### 
def print_id():
    id_list = []
    nums = len(gdnz) // len(sz)
    print(nums)
    for i in range(0, len(get_id())):
        for j in range(0, nums):
            id_list.extend(gsid[i])
    return id_list

In [15]:
pid = print_id()

40


### 输出文本并保存至csv

In [19]:
def get_data_news_content():
    lists, InfoPublDate, InfoSource, InfoTitle, InfoContent = [], [], [], [], []
    
    for i in range(0, len(gdnz)):                                                                    #gdnz是要抓取的新闻链接
        r_content = request.get(gdnz[i], 3)
        r_content.encoding = 'utf-8'
        selector = etree.HTML(r_content.text)
        info_contents = selector.xpath("//div[@class='article']")

        for info in info_contents:
            ###解析时间来员标题关键词文本
            InfoPublDate = info.xpath('//div[@class="date-source"]/span/text()')
            InfoSource = info.xpath('//div[@class="date-source"]/a/text()')
            InfoTitle = info.xpath('//h1/text()')
            InfoKeyword = info.xpath('//div[@id="article-bottom"]/div[@class="keywords"]/a/text()')
            InfoContent = info.xpath('//div[@class="article"]//p//text()')


            InfoPublDate.extend(InfoSource)
            InfoPublDate.extend(InfoTitle)
            Join_Key = ' '.join(InfoKeyword)
            InfoPublDate.append(Join_Key)
            Join_Con = ''.join(InfoContent)
            InfoPublDate.append(Join_Con)
            InfoPublDate.extend([gdnz[i]])
#             for j in range(3):
#                 InfoPublDate[j] = InfoPublDate[j].replace(u'\u3000',u'')

            ###文本合并StockID
            
            InfoPublDate.append(pid[i])
            
            with open('news_sina_2.csv', 'a+', encoding='utf-8-sig', newline='') as csvfile:
                w = csv.writer(csvfile, dialect='excel')
                w.writerow(InfoPublDate)        
    return InfoPublDate

In [21]:
gdnc = get_data_news_content()

## 保存csv至Mysql

In [22]:
import MySQLdb
import pymysql
import codecs

In [33]:
csv = pd.read_csv('news_sina_2.csv', header=None)

In [121]:
print(csv[0][1])

2018年09月11日 21:17


In [124]:
conn = MySQLdb.connect(host='localhost', user='root', passwd='231111', db='mysql', charset='utf8')
cur = conn.cursor()
for i in range(0, len(gdnz)):
    try:
        cur.execute("insert into sinainfo(InfoPublDate,InfoSource, InfoTitle, InfoKeyword, InfoContent, Url, StockID) values ('%s', '%s', '%s', '%s', '%s', '%s', '%s')" % (csv[0][i], csv[1][i], csv[2][i], csv[3][i], csv[4][i], csv[5][i], csv[6][i]))
        #stockid = cur.execute("insert into sinainfo() values ()" % )
        #print('date:', date)
        conn.commit()
    except Exception as e:
        #print(e)
        conn.rollback()

cur.close()


## 去重

In [83]:
def distinct():
    conn = MySQLdb.connect(host='localhost', user='root', passwd='231111', db='mysql', charset='utf8')
    cur = conn.cursor()
    sql_groupby = "DELETE from sinainfo where InfoPublDate NOT IN                                      \
                    (                                                                                  \
                        SELECT s.InfoPublDate from                                                     \
                            (                                                                          \
                            SELECT InfoPublDate from sinainfo where InfoPublDate GROUP BY InfoPublDate \
                        )AS s                                                                          \
                    )"
    try:
        cur.execute(sql_groupby)
        conn.commit()
        date = cur.fetchall()
        print('date:',date[4])
        
    except Exception as e:
        print('e:',e)
        conn.rollback()
cur.close()

In [84]:
distinct()

e: (1292, "Truncated incorrect INTEGER value: '2018年09月14日 07:21'")
